In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from openai import OpenAI
import json
import os
import numpy as np

In [2]:
def load_data(file_path):
    return pd.read_csv(file_path)


def filter_data(df, start_time, frame):
    def generate_range(start_time, frame):
        dtStart = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
        dtDiff = timedelta(hours=frame[0], minutes=frame[1])
        dest = dtStart + dtDiff
        return [start_time, dest.strftime('%Y-%m-%d %H:%M:%S')]
    
    time_range = generate_range(start_time, frame)
    return df[(df["time"] >= time_range[0]) & (df["time"] <= time_range[1])]

def initialize_api_client(api_key):
    return OpenAI(
        base_url="https://hackathon.radiantai.com/insight-ops/openai",
        api_key=api_key
    )


In [49]:
def extract_image_data(df_1, context, post_contents, image_index, api_client):
    post_orgininal = df_1['translation']
    imgs = df_1['attachment_urls'].split(",")
    print(imgs)
    if len(df_1['attachment_urls']) == 0:
        file_key = imgs[0]
    else:
        file_key = imgs[image_index]

    #Return false if the file ends with any extension that is not an image
    if not file_key.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')):
        return False
    
    image_url = "https://vl-nat-sec-hackathon-may-2024.s3.amazonaws.com/" + file_key

    system_prompt = """You are a helpful assistant designed to output JSON and analyze images. This image in the user message was attached to a post. We had a program analyze the post and come up with subjects and relationships/actions between those subjects. You will be provided that data of the subjects and relationships, the context of the post to mkae sure you do not mention anything irrelevant. You need to do the following:
    Analyze the image and provide the subjects and relationships/actions between those subjects to enter into the graph database.
    MAKE SURE to output the data in a JSON format with the top level keys 'subjects' and 'relationships', 
    Every object has a title and it has a type, the title indicates the name of the subject: this can be an person or "actor", event, or even concept, just make up a classification that is very standard.
    Two objects may be related in some way and this may and in this case, you will add them to the relationships list, try to find fitting relationships between related objects. For example, if you create a relationship between a person and an event you can use the relationship "participated in" or "attended". Or if you have a person and a concept you can use "knows" or "is familiar with". Or if you have two events you can use "happened before" or "happened after". Make up a relationship.
    How you will enter it in to the graph database it as follows, only include finding from the image in this response (meaning you will output in JSON format, DO NOT OUTPUT ANYTHING ELSE, IT WILL BREAK THE WHOLE PROGRAM):
    {
        "subjects": [
        [ENTITY_ID: INT, TITLE: STR, TYPE: STR],
        [ENTITY_ID: INT, TITLE: STR, TYPE: STR],
        ...
        ],
        "relationships": [
        [ENTITY_ID_1: INT, RELATIONSHIP: STR, ENTITY_ID_2: INT, PROPERTIES: DICT],
        [ENTITY_ID_1: INT, RELATIONSHIP: STR, ENTITY_ID_2: INT, PROPERTIES: DICT],
        ...
        ]
    }
    When you find a node or relationship you want to add try to create a generic PROPERTY for it that describes the entity you can also think of it as a label.
    Include things relevant to the""", context, """. 
    Also make sure that the RELATIONSHIP is consise, if the relationship is not simple to explain, it may not be relevant to the context of the""", context, """and should be left out. Analyze the image in keeping in mind the similarity and relevance of it to the QUALITIES OF THE PREVIOUS MOST HERE: """, str(json.dumps(post_contents)) , """and provide the JSON response, remember to look at the subjects of the post it is attached to, if there is something potentially related to a subject in the post, include it in the response. Here is the original post:""", post_orgininal, """"""
    print(image_url)
    response = api_client.chat.completions.create(
        model='gpt-4-vision-preview',
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': [{'type': 'image_url', 'image_url': {'url': image_url}}]}
        ]
    )
  
    return json.loads("".join((response.choices[0].message.content).split("\n")[1:-1]))

def process_post(post, api_client):
    
    system_prompt = """You are a helpful assistant designed to output JSON.
Your task is to extract information from social media posts or announcements relevant to the Ukraine Russian war and convert it into a graph database.

Here is how the graph database works:
Every object has a title and it has a type, the title indicates the name of the subject: this can be an person or "actor", event, or even concept, just make up a classification that is very standard.
Two objects may be related in some way and this may and in this case, you will add them to the relationships list, try to find fitting relationships between related objects. For example, if you create a relationship between a person and an event you can use the relationship "participated in" or "attended". Or if you have a person and a concept you can use "knows" or "is familiar with". Or if you have two events you can use "happened before" or "happened after". Make up a relationship.
How you will enter it in to the graph database it as follows:
{
  "subjects": [
    [ENTITY_ID: INT, TITLE: STR, TYPE: STR],
    [ENTITY_ID: INT, TITLE: STR, TYPE: STR],
    ...
  ],
  "relationships": [
    [ENTITY_ID_1: (ANY), RELATIONSHIP: STR, ENTITY_ID_2: (ANY), PROPERTIES: DICT],
    [ENTITY_ID_1: (ANY), RELATIONSHIP: STR, ENTITY_ID_2: (ANY), PROPERTIES: DICT],
    ...
  ]
}
If you can't pair a relationship with a pair of nodes don't add it.
When you find a node or relationship you want to add try to create a generic PROPERTY for it that describes the entity you can also think of it as a label.
Only include things relevant to the Ukraine Russia conflict
One more thing: this image IS ATTACHED TO A POST, so try to find a relationship between the image subjects and the post subjects. This image needs to add to the actual post subjects and relationships, as it is attached to the post."""

    
    response = api_client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Here's the post: {post} \\n Make sure to output in a JSON with the top level keys 'subjects' and 'relationships'"}
        ]
    )
    return json.loads(response.choices[0].message.content)


def extract_graph_data(df, api_client):
    """
    Iterate over a DataFrame of posts, process each one, and accumulate the graph data.
    
    Args:
    df (pandas.DataFrame): DataFrame containing the posts to process.
    api_client (OpenAI.Client): The API client instance.
    
    Returns:
    dict: A dictionary containing accumulated nodes and edges from all posts.
    """
    system_prompt = "Your task is to extract information from social media posts and convert it into a graph database."
    all_data = {'subjects': [], 'relationships': []}  # Initialize structure to hold combined data from all posts
    
    img_data = {}

    image_data = []

    for index, row in df.iterrows():
        
        post_data = process_post(row['translation'], api_client)

        image_data = []
        for i in range(len(row["attachment_urls"].split(","))):
            try:
                image_1 = extract_image_data(row, "Russia vs. Ukraine", post_data, i, api_client)
            except:
                image_1 = False
            if image_1:
                image_data.append(image_1)
                post_data['subjects'].extend(image_1['subjects'])
                post_data['relationships'].extend(image_1['relationships'])
        
        try:
            for x in post_data['subjects']:
                x[0] = row['id'] + '_' + str(x[0])

            for x in post_data['relationships']:
                x[0] = row['id'] + '_' + str(x[0])
                x[2] = row['id'] + '_' + str(x[2])
        except:
            continue
        else:
            all_data['subjects'].extend(post_data['subjects'])
            all_data['relationships'].extend(post_data['relationships'])
    
    return all_data

In [341]:
'''

def deduping_nodes(nodes, api_client):
    
    system_prompt = """You are a helpful assistant designed to do data disambiguation nodes from a knowledge graph about the Russia Ukraine war.

Here is how the graph database works:
Every object has a title and it has a type, the title indicates the name of the subject
The input is a list of lists and this is what the input looks like:
[
    [ENTITY_ID: INT, TITLE: STR, TYPE: STR],
    [ENTITY_ID: INT, TITLE: STR, TYPE: STR],
    ...
  ]

You have to output a list of tuples for the titles that are different entries in the list but actually represent the same entity.

Example of entities that are the same could be [(Russian Army, Russian Armed Forces)]"""

    system_prompt = """Your task is to identify if there are duplicated nodes in a noisy knowledge graph 
    
You will be given a datasets of nodes and some of these nodes may be duplicated or refer to the same entity. 
The datasets contains nodes in the form [ENTITY_ID, ENTITY_NAME, TYPE]. 

There are many duplicates in this dataset with slightly different phraseology for the ENTITY_NAME

When you have completed your task please give me the pairwise entities that are reasonably the duplicate entities. 

The output should be in the format [(a,b)] 
"""
    response = api_client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Here's the input: {nodes}. Make sure the output is in JSON \\n"}
        ]
    )
    return json.loads(response.choices[0].message.content)

deduped = deduping_nodes(structured_data["subjects"][:20], api_client)
deduped
'''

In [6]:
api_key = "rad-69912515feb6baab52a0c1fcf1c3d283cd29b9af1e21ed8117158eb54e338e41-iant"#os.environ["RADIANT_API_KEY"]
df = load_data('data/russia_social_media.csv')


In [54]:
filtered_df = filter_data(df, "2024-03-31 18:00:00", [2, 0])
api_client = initialize_api_client(api_key)
filtered_df.shape

(665, 9)

In [46]:
with open('raw_posts_300.json', 'w') as f:
    f.write(filtered_df.set_index('id').to_json(orient='index', indent=4))

In [ ]:
%%time
structured_data = extract_graph_data(filtered_df, api_client)
len(structured_data['subjects'])

['media/telegram-web/157cfb52-f4b1-407c-ad3f-6a3c4c4a68a4.jpeg']
https://vl-nat-sec-hackathon-may-2024.s3.amazonaws.com/media/telegram-web/157cfb52-f4b1-407c-ad3f-6a3c4c4a68a4.jpeg
['media/telegram-web/24928fd5-c796-4a6b-83cf-1fa1078017ee.mp4', 'kirby/telegram-web/9fca1fb4-c27a-495e-a630-dac8634f2794.jpeg']
['media/telegram-web/24928fd5-c796-4a6b-83cf-1fa1078017ee.mp4', 'kirby/telegram-web/9fca1fb4-c27a-495e-a630-dac8634f2794.jpeg']
https://vl-nat-sec-hackathon-may-2024.s3.amazonaws.com/kirby/telegram-web/9fca1fb4-c27a-495e-a630-dac8634f2794.jpeg
['media/telegram-web/4012b8bb-bd80-4789-a63f-9c2abb9b3d13.jpeg']
https://vl-nat-sec-hackathon-may-2024.s3.amazonaws.com/media/telegram-web/4012b8bb-bd80-4789-a63f-9c2abb9b3d13.jpeg
['media/telegram-web/976bd463-9a73-44a7-938b-29327fdf3ce8.jpeg']
https://vl-nat-sec-hackathon-may-2024.s3.amazonaws.com/media/telegram-web/976bd463-9a73-44a7-938b-29327fdf3ce8.jpeg
['media/telegram-web/984798ed-cdd8-48e8-bb28-0ddb9f3b1d6b.jpeg']
https://vl-nat-sec-h

['media/telegram-web/89a12205-0afa-4054-82b3-161ad640d187.mp4', 'kirby/telegram-web/ac877f76-7393-44fb-a8b8-6a0174954d0d.jpeg']
['media/telegram-web/89a12205-0afa-4054-82b3-161ad640d187.mp4', 'kirby/telegram-web/ac877f76-7393-44fb-a8b8-6a0174954d0d.jpeg']
https://vl-nat-sec-hackathon-may-2024.s3.amazonaws.com/kirby/telegram-web/ac877f76-7393-44fb-a8b8-6a0174954d0d.jpeg
['media/telegram-web/e28d241d-185d-4295-9b3a-28ee9bc21fa0.mp4', 'kirby/telegram-web/00e48c38-725b-4b56-9ca0-7621bfdce17f.jpeg']
['media/telegram-web/e28d241d-185d-4295-9b3a-28ee9bc21fa0.mp4', 'kirby/telegram-web/00e48c38-725b-4b56-9ca0-7621bfdce17f.jpeg']
https://vl-nat-sec-hackathon-may-2024.s3.amazonaws.com/kirby/telegram-web/00e48c38-725b-4b56-9ca0-7621bfdce17f.jpeg
['media/telegram-web/62b67885-86cf-400d-adcb-4b94d441d5d0.mp4', 'kirby/telegram-web/42096e96-f75d-4040-904e-603fa43ad55d.jpeg']
['media/telegram-web/62b67885-86cf-400d-adcb-4b94d441d5d0.mp4', 'kirby/telegram-web/42096e96-f75d-4040-904e-603fa43ad55d.jpeg']


In [27]:
with open('structured_data_300.json') as f:
    structured_data = json.load(f)

In [28]:
def create_graph(data):
    G = nx.DiGraph()
    for node in data['subjects']:
        node_id, name, node_type = node
        G.add_node(node_id, name = name, node_type = node_type)
        
    for edge in data['relationships']:
        try:
            source, relation, target, props = edge
        except:
            continue
        else:
            G.add_edge(source, target, relation=relation)
    return G

def draw_graph(G):
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, node_size=100, node_color='lightblue', alpha=0.6)
    nx.draw_networkx_edges(G, pos, arrowstyle='-|>', arrowsize=5, edge_color='gray', width=2)
    node_labels = {node: G.nodes[node]['name'] for node in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=12)
    edge_labels = {(u, v): data['relation'] for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    plt.title('Graph Visualization')
    plt.axis('off')
    plt.show()

In [29]:
G = create_graph(structured_data)

In [31]:
import difflib
from collections import defaultdict

# Function to group nodes by name similarity
def group_similar_nodes(graph, cutoff=0.8):
    names = nx.get_node_attributes(graph, 'name')
    nodes = sorted(names, key=lambda x: names[x])
    clusters = []
    used = set()

    for node in nodes:
        if node not in used:
            cluster = {node}
            for other in nodes:
                if other not in used and difflib.SequenceMatcher(None, names[node], names[other]).ratio() >= cutoff:
                    cluster.add(other)
            clusters.append(cluster)
            used.update(cluster)

    return clusters

# Function to merge nodes
def merge_nodes(graph, clusters):
    new_graph = nx.DiGraph()
    for cluster in clusters:
        # Concatenate names of all nodes in the cluster
        merged_name = ' ; '.join(set(graph.nodes[n]['name'] for n in cluster))
        node_type = ' ; '.join(set(graph.nodes[n]['node_type'] for n in cluster))
        new_node = min(cluster)  # Use the smallest node ID in the cluster as the new node ID

        new_graph.add_node(new_node, name=merged_name, node_type = node_type)

        # Merging edges and their attributes
        edge_attributes = defaultdict(set)
        for old_node in cluster:
            for neighbor, attrs in graph[old_node].items():
                if neighbor not in cluster:
                    edge_attributes[(new_node, neighbor)].add(attrs['relation'])

        # Add merged edges with combined attributes to the new graph
        for (u, v), relations in edge_attributes.items():
            new_graph.add_edge(u, v, relation=list(relations))

    return new_graph

# Find clusters of similar nodes and merge them
clusters = group_similar_nodes(G)
new_G = merge_nodes(G, clusters)


In [35]:
nx.write_gml(new_G, 'graph_300.gml')

In [47]:
def graph_to_custom_json(graph):
    # Only include nodes that have a 'name' attribute
    subjects = [[node, graph.nodes[node]['name'], 'single_type'] for node in graph.nodes() if 'name' in graph.nodes[node]]
    
    # Prepare relationships, making sure both nodes in the edge have 'name' attributes
    relationships = [[u, graph.edges[u, v]['relation'], v, graph.edges[u, v].get('properties', {})]
                     for u, v in graph.edges() if 'name' in graph.nodes[u] and 'name' in graph.nodes[v]]

    return {
        "subjects": subjects,
        "relationships": relationships
    }

# Assuming 'new_G' is your graph object post-processing
custom_data = graph_to_custom_json(new_G)

# Save the JSON object to a file
with open('graph_300.json', 'w') as f:
    json.dump(custom_data, f, indent=4)

print("Graph saved in custom JSON format.")

Graph saved in custom JSON format.


In [33]:
new_G

In [19]:
def compute_graph_metrics(graph):
    # Ensure the graph is a directed graph; adapt as necessary for undirected graphs
    if not graph.is_directed():
        raise ValueError("Input graph must be directed. For undirected graphs, modify the function accordingly.")

    # Calculate metrics
    in_degrees = dict(graph.in_degree())  # In-degree
    out_degrees = dict(graph.out_degree())  # Out-degree
    try:
        eigenvector_centrality = nx.eigenvector_centrality_numpy(graph)  # Eigenvector centrality
    except nx.NetworkXError as e:
        print("Error calculating eigenvector centrality:", e)
        eigenvector_centrality = {n: 0 for n in graph.nodes()}  # Default to 0 if calculation fails

    # Component ID assignment using weakly connected components
    components = list(nx.weakly_connected_components(graph))
    component_dict = {node: idx for idx, component in enumerate(components) for node in component}

    # Create DataFrame
    data = {
        'Node': list(graph.nodes()),
        'In-Degree': [in_degrees.get(node) for node in graph.nodes()],
        'Out-Degree': [out_degrees.get(node) for node in graph.nodes()],
        'Eigenvector Centrality': [eigenvector_centrality.get(node, 0.0) for node in graph.nodes()],
        'Component ID': [component_dict.get(node, -1) for node in graph.nodes()]
    }
    df = pd.DataFrame(data)
    return df

In [39]:
with open('graph_300_nodes.json', 'w') as file:
    file.write(compute_graph_metrics(new_G).set_index('Node').to_json(orient='index', indent=4))



In [48]:
def get_scaled_positions(graph):
    # Calculate optimal spacing between nodes
    size_factor = 40  # Maximum node size
    k_value = size_factor / 10  # k proportional to node size

    # Use the spring layout to get initial positions
    pos = nx.spring_layout(graph, k=k_value, iterations=100)  # More iterations for a better layout

    # Scale the positions up to further ensure no overlap
    pos_scaled = {node: (x * 100, y * 100) for node, (x, y) in pos.items()}
    # Convert positions to a format that can be serialized to JSON
    pos_json_ready = {node: {'x': pos[0], 'y': pos[1]} for node, pos in pos_scaled.items()}


    return pos_json_ready


# Save the positions to a JSON file
with open('graph_300_positions.json', 'w') as json_file:
    json.dump(get_scaled_positions(new_G), json_file, indent=4)

{'subjects': [['0007ed21-eb78-597d-9558-9dfa4f75ed3e_1', 'Oles Buzina', 'person'], ['0007ed21-eb78-597d-9558-9dfa4f75ed3e_2', 'Vladimir Zhirinovsky', 'person'], ['0007ed21-eb78-597d-9558-9dfa4f75ed3e_3', 'Vladimir Solovyov', 'person'], ['0007ed21-eb78-597d-9558-9dfa4f75ed3e_4', 'Duel TV Talk Show', 'event'], ['0124953a-08ac-53d7-963b-8eacbeb43150_1', 'Russians', 'actor'], ['0124953a-08ac-53d7-963b-8eacbeb43150_2', 'Kharkiv region', 'concept'], ['0124953a-08ac-53d7-963b-8eacbeb43150_3', "152-mm self-propelled gun 2C19 'Msta-S'", 'actor'], ['0124953a-08ac-53d7-963b-8eacbeb43150_4', 'Volyn boys', 'actor'], ['0124953a-08ac-53d7-963b-8eacbeb43150_5', '14th separate mechanized brigade', 'actor'], ['0124953a-08ac-53d7-963b-8eacbeb43150_6', 'Prince Roman the Great', 'actor'], ['0124953a-08ac-53d7-963b-8eacbeb43150_7', 'small drone', 'actor'], ['016965b7-2511-5a9b-adb1-f5b9415c6903_1', 'Biden', 'person'], ['016965b7-2511-5a9b-adb1-f5b9415c6903_2', 'Platform X', 'social media platform'], ['01696